In [1]:
from misc import load, load_class_labels

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train_info_cleaned.csv', 1, sample_rate)
test_info = load(dataset_root + 'labels/test.csv')
val_info = load(dataset_root + 'labels/val_info_cleaned.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [2]:
from cnn_classification import build_finetune_network, train_network
from tensorflow import keras
from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input

extension = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(251, activation='softmax')])

loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']

cut_layer = 'top_dropout'
non_trainable_cut_layer = 'block5e_add'

extended_model = build_finetune_network(EfficientNetV2B0, extension, cut_layer,
                                        optimizer, metrics, loss, non_trainable_cut_layer=non_trainable_cut_layer)


save_path = "./models/efficientnet_v2_standard"
train_loss, train_accuracy, val_loss, val_accuracy = train_network(extended_model,
                                                                   save_path,
                                                                   train_info,
                                                                   val_info,
                                                                   train_dir,
                                                                   epochs=10,
                                                                   augment=True,
                                                                   strong_augment=False,
                                                                   batch_size=64,
                                                                   preprocess_input=preprocess_input)

2024-01-01 15:59:26.392516: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 15:59:26.392545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 15:59:26.395644: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 15:59:26.451205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 15:59:27.434837: W tensorflow/compiler/tf2

Epoch 1/10 iteration 1/1159 Loss:     5.5823 Accuracy:     0.0000

2024-01-01 15:59:58.066530: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 38535168 exceeds 10% of free system memory.
2024-01-01 15:59:58.112880: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 38535168 exceeds 10% of free system memory.


Epoch 1/10 iteration 1158/1159 Loss:     3.4802 Accuracy:     0.3167

2024-01-01 16:11:36.874857: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/model/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 1/10 iteration 1159/1159 Loss:     3.4801 Accuracy:     0.3167 Validation Loss:     3.1423, Validation accuracy:     0.4096
INFO:tensorflow:Assets written to: ./models/efficientnet_v2_standard/assets


INFO:tensorflow:Assets written to: ./models/efficientnet_v2_standard/assets
2024-01-01 16:14:37.664836: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/model/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 2/10 iteration 90/1159 Loss:     2.7990 Accuracy:     0.4344

KeyboardInterrupt: 

In [ ]:
EfficientNetV2B0().summary()

In [ ]:
extended_model.summary()

In [ ]:
extended_model.save('./finetune_network/model')
from tensorflow import keras
keras.applications.EfficientNetV2B0().summary()

In [ ]:
from tensorflow import keras
from cnn_classification import train_network
import gc

path = './efficientnet_noise'
for i in range(20):
    extended_model = keras.models.load_model(path)
    train_network(extended_model, train_info, val_info, train_dir, epochs=1, augment=True, batch_size=64, preprocess_input=preprocess_input)
    extended_model.save(path)
    del extended_model
    gc.collect()
    keras.backend.clear_session()

In [ ]:
from tensorflow import keras
from cnn_classification import evaluate_model, predict

model = keras.models.load_model(path)
predictions = predict(model, val_degraded_dir, 64, preprocess_input=preprocess_input)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

num_pred = []
for p in predictions:
    num_pred.append(np.argmax(p))
print(len(predictions), len(test_labels))

cm = confusion_matrix(test_labels, num_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

In [ ]:
cm

In [ ]:
cm.diagonal().sum() / cm.sum()